In [ ]:
import torch
import numpy as np
from numpy.linalg import inv
from numpy.linalg import norm
import matplotlib.pyplot as plt

from random import seed
from random import random

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
np.random.seed(42)

w1 = w11 = np.random.rand()
w3 = -w11
w2 = w12 = np.random.rand()
w4  = -w12
w5 = w21 = np.random.rand()
w7 = -w21
w6 = w22 = np.random.rand()
w8 = -w22

a=np.array([[w1,w2],[w3,w4],[w5,w6],[w7,w8]])
# print(a)


In [ ]:
# This funtion generates data 
As = []
for condition_number in [1,0.1,0.01,0.0001]: # create a list of As with different conditional number
        A = np.matrix([[1, 1], [1, 1+condition_number]]) 
        As.append(A)
        
def data_generator(n):
    #np.random.seed(42)
    
#     As = []
#     for condition_number in [1,0.1,0.01,0.0001]: # create a list of As with different conditional number
#         A = np.matrix([[1, 1], [1, 1+condition_number]]) 
#         As.append(A)
 
    x = np.random.normal(0,1,size = (2,n)) # n is the size of sample
    noise = np.random.normal(0,0.04**2) 
        
    y1 = np.dot(As[0],x) + 0.01*noise #compute y with different A
    y01 = np.dot(As[1],x) + 0.01*noise
    y001 = np.dot(As[2],x) + 0.01*noise
    y0001 = np.dot(As[3],x) + 0.01*noise
    
    x = torch.Tensor(x).float() #convert all ndarrays to tensors
    y1 = torch.Tensor(y1).float()
    y01 = torch.Tensor(y01).float()
    y001 = torch.Tensor(y001).float()
    y0001 = torch.Tensor(y0001).float()
#     print(x.size())
#     print(y1.size())
    
    return x, y1, y01, y001, y0001    

In [ ]:
def tikhonov(n):
    train_x_all, train_y_1,train_y_01,train_y_001,train_y_0001= data_generator(n)
    print("train_x_all-----",train_x_all)
    print("train_y_1-----",[train_y_1,train_y_01,train_y_001,train_y_0001])

    for index,Y in enumerate([train_y_1,train_y_01,train_y_001,train_y_0001]):
        globals()['x_tik_'+str(index)] = np.matmul(np.matmul(inv(np.matmul(As[index].T,As[index])+0.04**2*np.eye(2)),As[index].T),Y)
    
    for index,X_tik in enumerate([x_tik_0,x_tik_1,x_tik_2,x_tik_3]):
        globals()['Error_x_tik_'+str(index)] = (torch.norm(train_x_all-X_tik))**2 * (1./n)
    
    print("Error_tik_1-----",Error_x_tik_0)
    print("Error_tik_01-----",Error_x_tik_1)
    print("Error_tik_001-----",Error_x_tik_2)
    print("Error_tik_0001-----",Error_x_tik_3)
    return x_tik_0,x_tik_1,x_tik_2,x_tik_3

In [ ]:
tikhonov(10)

In [ ]:
def Relu(Z):
    return np.maximum(0,Z)

def dRelu(x):
    return np.where(x > 0, 1.0, 0.0)

In [ ]:
class net:
    def __init__(self, x, y):
        self.X=x
        self.Y=y
        self.Yh=np.zeros((1,self.Y.shape[1]))
        self.weights = {}
        self.ch = {} # a cache variable
        self.loss = []
        self.lr=0.03
        self.size = self.Y.shape[1]
    def nInit(self):    
        np.random.seed(1)
        self.weights['W1'] = torch.from_numpy(np.array([[w1,w2],[w3,w4],[w5,w6],[w7,w8]]))  #(4,2)  
        self.weights['W2'] = torch.from_numpy(np.array([[1,-1,0,0],[0,0,1,-1]]))  #(2,4)   
        return

    def forward(self):    
        Z1 = np.matmul(self.weights['W1'],self.X) #self.weights['W1'].dot(self.X)
        A1 = Relu(Z1)
        self.ch['Z1'],self.ch['A1']=Z1,A1
        
        Z2 = np.matmul(self.weights['W2'],A1) #self.weights['W2'].dot(A1)
        self.ch['Z2'] = Z2
        self.Yh=Z2
        loss=self.calculate_loss(Z2)
        return self.Yh, loss
    
    def calculate_loss(self,Yh):
        loss = (torch.norm(self.Y-self.Yh))**2 * (1./self.size) #norm
        return loss
    
    def backward(self):
        #Divide the Yh matrix and the X matrix into 2 rows, and assign them to different variables
        Yh1 = self.Yh[0] #(1xn)matrix size
        Yh2 = self.Yh[1] #(1xn)
        X1 = self.X[0] #(1xn)
        X2 = self.X[1] #(1xn)
        
        dLoss_Yh = -2*(self.Y - self.Yh)* (1./self.size) #(2,n)
        dloss_yh1 =  dLoss_Yh[0]
        dloss_yh2 = dLoss_Yh[1]

        
        dyh1_A1 = dyh2_A3 = 1
        dyh1_A2 = dyh2_A4 = -1
        dyh1_A3 = dyh2_A1 = 0
        dyh1_A4 = dyh2_A2 = 0

        
        dA1_w1 = torch.from_numpy(dRelu(self.ch['Z1'][0]))
        dA2_w3 = torch.from_numpy((dRelu(self.ch['Z1'][0])-1))
        
        dA1_w2 = torch.from_numpy(dRelu(self.ch['Z1'][0]))
        dA2_w4 = torch.from_numpy((dRelu(self.ch['Z1'][0])-1))
        
        dA3_w5 = torch.from_numpy(dRelu(self.ch['Z1'][2]))
        dA4_w7 = torch.from_numpy(dRelu(self.ch['Z1'][2])-1)
        
        dA3_w6 = torch.from_numpy(dRelu(self.ch['Z1'][2]))
        dA4_w8 = torch.from_numpy(dRelu(self.ch['Z1'][2])-1)
        
        dloss_w1 = np.matmul(dloss_yh1*(dyh1_A1*dA1_w1+dyh1_A2*dA2_w3),X1.T)
        dloss_w2 = np.matmul(dloss_yh1*(dyh1_A1*dA1_w2+dyh1_A2*dA2_w4),X2.T)
        dloss_w5 = np.matmul(dloss_yh2*(dyh2_A3*dA3_w5+dyh2_A4*dA4_w7),X1.T)
        dloss_w6 = np.matmul(dloss_yh2*(dyh2_A3*dA3_w6+dyh2_A4*dA4_w8),X2.T)

        dloss_W1 = torch.from_numpy(np.array([[dloss_w1,dloss_w2],[-dloss_w1,-dloss_w2],[dloss_w5,dloss_w6],[-dloss_w5,-dloss_w6]]))  #(4,2)

        self.weights["W1"] = self.weights["W1"] - self.lr * dloss_W1
    
    def predict(self, test_data):
        prediction = np.matmul(self.weights['W2'],Relu(np.matmul(self.weights['W1'],test_data)))
        return self.calculate_loss(prediction)
    
    def run(self,X, Y, epochs):
        self.loss = []
        self.weightlist = []
        np.random.seed(1)                         
        self.nInit()
        for i in range(0, epochs):
            Yh, loss=self.forward()
            self.backward()
            self.loss.append(loss)
            if i >= 300:
                self.weightlist.append(self.weights["W1"].numpy())
                
#         print("-"*50)
#         print("W1",self.weights["W1"],"\n")
#         print("Y",Y)
#         print("Yh",Yh)

        return self.loss,self.weightlist
        
    

In [ ]:
def main(n,epochs):
    
    np.random.seed(42)
    train_x_all, train_y_1,train_y_01,train_y_001,train_y_0001= data_generator(n) # create the data 
    test_x_all, test_y_1,test_y_01,test_y_001,test_y_0001=data_generator(n)
    
    
    nn = net(train_x_all,train_y_1)
    list_1,w_list_1 = nn.run(train_x_all,train_y_1,epochs)
    print("test_1_NMSE-----",nn.predict(test_y_1))
#     print(list_1)

    nn = net(train_x_all,train_y_01)
    list_01,w_list_01 = nn.run(train_x_all,train_y_01,epochs)
    print("test_01_NMSE-----",nn.predict(test_y_01))
#     print(list_01)

    nn = net(train_x_all,train_y_001)
    list_001,w_list_001 = nn.run(train_x_all,train_y_001,epochs)
    print("test_001_NMSE-----",nn.predict(test_y_001))
#     print(list_001)

    nn = net(train_x_all,train_y_0001)
    list_0001,w_list_0001 = nn.run(train_x_all,train_y_0001,epochs)
    print("test_0001_NMSE-----",nn.predict(test_y_0001))
    
    for w in [w_list_1,w_list_01,w_list_001,w_list_0001]:
        mean = np.sum(w,axis=0)/len(w)
        mean0fsquared_w = np.sum(np.square(w),axis=0)/len(w)
        
        print("~"*50)
        print("MEAN------",mean)
        print("VARIENCE------",mean0fsquared_w-np.square(mean))
    
    # plot 3000 epochs and the losses
    epochs = 3000
    plt.suptitle('Forward Problem')
    plt.plot(range(epochs),list_1)
    plt.plot(range(epochs),list_01)
    plt.plot(range(epochs),list_001)
    plt.plot(range(epochs),list_0001)
#     x1,x2,y1,y2 = plt.axis()
#     plt.axis((x1,x2,0,14))
    plt.xlabel('Number of Epochs')
    plt.ylabel('Normalised Mean Squared Error')
    plt.legend(['1', '0.1', '0.01', '0.0001'], loc='upper right')
    plt.show()

#     plot the first 1000 epochs
    epochs = 1000
    plt.suptitle('Forward Problem')
    plt.plot(range(epochs),list_1[:1000])
    plt.plot(range(epochs),list_01[:1000])
    plt.plot(range(epochs),list_001[:1000])
    plt.plot(range(epochs),list_0001[:1000])
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0,0.1))
    plt.xlabel('Number of Epochs')
    plt.ylabel('Normalised Mean Squared Error')
    plt.legend(['1', '0.1', '0.01', '0.0001'], loc='upper right')
    plt.show()

In [ ]:
def inverse_main(n,epochs):
    np.random.seed(42)
    
    train_x_all, train_y_1,train_y_01,train_y_001,train_y_0001= data_generator(n) # create the data 
    test_x_all, test_y_1,test_y_01,test_y_001,test_y_0001=data_generator(n)
    
    nn = net(train_y_1, train_x_all)
    list_1,w_list_1 = nn.run(train_y_1, train_x_all,epochs)
    #print("test_1_NMSE-----",nn.predict(test_x_all))
    
    x_t1 = train_x_all[0]
    x_t2 = train_x_all[1]
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]

    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.suptitle('The distribution of the estimated x (epsilon=1) - random data')
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x', 'estimated x'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()

    nn = net(train_y_01, train_x_all)
    list_01,w_list_01 = nn.run(train_y_01, train_x_all,epochs)
    #print("test_01_NMSE-----",nn.predict(test_x_all))
    
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x (epsilon=0.1) - random data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x', 'estimated x'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    nn = net(train_y_001, train_x_all)
    list_001,w_list_001 = nn.run(train_y_001, train_x_all,epochs)
    #print("test_001_NMSE-----",nn.predict(test_x_all))
    
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x (epsilon=0.01) - random data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x', 'estimated x'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    nn = net(train_y_0001, train_x_all)
    list_0001,w_list_0001 = nn.run(train_y_0001, train_x_all,epochs)
    #print("test_0001_NMSE-----",nn.predict(test_x_all))
    
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x (epsilon=0.0001) - random data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x', 'estimated x'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    print("The mean and varience of the pseudo-inverse -stmmetric network with random data")
    for w in [w_list_1,w_list_01,w_list_001,w_list_0001]:
        mean = np.sum(w,axis=0)/len(w)
        mean0fsquared_w = np.sum(np.square(w),axis=0)/len(w)
        
        print("~"*50)
        print("MEAN------",mean)
        print("VARIENCE------",mean0fsquared_w-np.square(mean))
        
    # plot 3000 epochs and the losses
    epochs = 3000
    plt.suptitle('Inverse Problem - python')
    plt.plot(range(epochs),list_1)
    plt.plot(range(epochs),list_01)
    plt.plot(range(epochs),list_001)
    plt.plot(range(epochs),list_0001)
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0,14))
    plt.xlabel('Number of Epochs')
    plt.ylabel('Normalised Mean Squared Error')
    plt.legend(['1', '0.1', '0.01', '0.0001'], loc='upper right')
    plt.show()

In [ ]:
main(10000,3000)

In [ ]:
inverse_main(10000,3000)

In [ ]:
def tick_inverse_main(n,epochs):
    np.random.seed(42)
    
    train_x_all, train_y_1,train_y_01,train_y_001,train_y_0001= data_generator(n) # create the data 
    
    for index,Y in enumerate([train_y_1,train_y_01,train_y_001,train_y_0001]):
        globals()['x_tik_'+str(index)] = np.matmul(np.matmul(inv(np.matmul(As[index].T,As[index])
                                                                 +0.04**2*np.eye(2)),As[index].T),Y)
        
    
    nn = net(train_y_1,x_tik_0)
    list_1,w_list_1 = nn.run(train_y_1,x_tik_0,epochs)
    
    x_t1 = x_tik_0[0]
    x_t2 = x_tik_0[1]
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x_tik (epsilon=1) - regularised data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x_tik', 'estimated x_tik'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    nn = net(train_y_01, x_tik_1)
    list_01,w_list_01 = nn.run(train_y_01, x_tik_1,epochs)
    
    x_t1 = x_tik_1[0]
    x_t2 = x_tik_1[1]
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x_tik (epsilon=0.1) - regularised data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x_tik', 'estimated x_tik'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    nn = net(train_y_001, x_tik_2)
    list_001,w_list_001 = nn.run(train_y_001, x_tik_2,epochs)
    #print(list_001)
    
    x_t1 = x_tik_2[0]
    x_t2 = x_tik_2[1]
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x_tik (epsilon=0.01) - regularised data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x_tik', 'estimated x_tik'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    nn = net(train_y_0001, x_tik_3)
    list_0001,w_list_0001 = nn.run(train_y_0001, x_tik_3,epochs)  
    
    x_t1 = x_tik_3[0]
    x_t2 = x_tik_3[1]
    x1= nn.Yh.numpy()[0]
    x2= nn.Yh.numpy()[1]
    plt.suptitle('The distribution of the estimated x_tik (epsilon=0.0001) - regularised data')
    plt.scatter(x_t1,x_t2,color="blue",alpha=0.2)
    plt.scatter(x1,x2,color="red",alpha=0.2)
    plt.legend(['original x_tik', 'estimated x_tik'], loc='upper right')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    print("The mean and varience of the pseudo-inverse -stmmetric network with regularised data")
    for w in [w_list_1,w_list_01,w_list_001,w_list_0001]:
        mean = np.sum(w,axis=0)/len(w)
        mean0fsquared_w = np.sum(np.square(w),axis=0)/len(w)
        
        print("~"*50)
        print("MEAN------",mean)
        print("VARIENCE------",mean0fsquared_w-np.square(mean))

    # plot 3000 epochs and the losses
    epochs = 3000
    plt.suptitle('Inverse Problem(tikhonov)- python')
    plt.plot(range(epochs),list_1)
    plt.plot(range(epochs),list_01)
    plt.plot(range(epochs),list_001)
    plt.plot(range(epochs),list_0001)
    x1,x2,y1,y2 = plt.axis()
#     plt.axis((x1,x2,0,14))
    plt.xlabel('Number of Epochs')
    plt.ylabel('Normalised Mean Squared Error')
    plt.legend(['original x_tik', 'estimated x_tik'], loc='upper right')
    plt.show()

In [ ]:
for index,A in enumerate(As):
        globals()['tick_approx_'+str(index)] = np.matmul(inv(np.matmul(As[index].T,As[index])+0.04**2*np.eye(2)),As[index].T)

In [ ]:
print("tick_approx_1",tick_approx_0,"\n")
print("tick_approx_0.1",tick_approx_1,"\n")
print("tick_approx_0.01",tick_approx_2,"\n")
print("tick_approx_0.0001",tick_approx_3,"\n")

In [ ]:
tick_inverse_main(10000,3000)